# Energy Plots
for whole inferences

In [1]:
%run '00_Imports-and-settings.ipynb'

Numpy Version:		 1.19.0
Pandas Version:		 1.0.5
Matplotlib Version:	 3.2.2


In [2]:
# helper function to get model name
def get_model_name(model_string):
    if 'ResNet' in model_string:
        model_name = 'ResNet20_CIFAR-10'
    elif 'LeNet' in model_string:
        model_name = 'LeNet-MNIST'
    else:
        model_name = 'unknown'
        print('unknown model name')
    return model_name

In [3]:
df = pd.read_pickle('results_energy/A_Aggregated_energy_inference_table.pkl')

In [4]:
df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

In [5]:
di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

In [6]:
df.columns

Index(['time', 'filename', 'MCU', 'model', 'model_name', 'mbed-dir',
       'cmsis-nn', 'compiler_optimization', 'FPU_status', 'model_type',
       'weights', 'activations', 'pruned', 'no_correct_inferences',
       'latency_mean', 'latency_std', 'voltage_mean', 'voltage_std',
       'current_mean', 'current_std', 'power_mean', 'power_std', 'energy_mean',
       'energy_std'],
      dtype='object')

In [7]:
df = df.reset_index()

## Cross-verify benchmarking times

In [8]:
df_energy = pd.read_pickle('results_energy/A_Aggregated_energy_inference_table.pkl')
df_benchmark = pd.read_pickle('results/A_aggregated_benchmarking+verification_results.pkl')


# loop over architecture, mcu, model_type, cmsis-nn

mcus = df_energy['MCU'].unique()
models = df_energy['model_name'].unique()


for model in models:
    for mcu in mcus:
        filter = (df_energy['weights'] == df_energy['activations']) & (df_energy['MCU'] == mcu) & (df_energy['model_name'] == model)
        df_filtered_energy = df_energy.loc[filter]
        
        df_filtered_energy = df_filtered_energy.sort_values(by=['model_type', 'cmsis-nn'])

        
        filter = (df_benchmark['compiler_optimization'] == '-Ofast') & (df_benchmark['FPU_status'] == 1) & \
                (df_benchmark['weights'] == df_benchmark['activations']) & (df_benchmark['MCU'] == mcu) & (df_benchmark['model'].str.contains(model))
        
        #print((df_benchmark['MCU'] == mcu))
        #print((df_energy['model'].str.contains(model)))
        df_filtered_benchmark = df_benchmark.loc[filter]
        
        df_filtered_benchmark = df_filtered_benchmark.sort_values(by=['model_type', 'cmsis-nn'])


#         try:
#             df_filtered_energy['tflu_benchmark'] = df_filtered_benchmark['tflu_mcu_benchmark_mean_50'].values
#         except KeyError:
#             df_filtered_energy['tflu_benchmark'] = df_filtered_benchmark['tflu_mcu_benchmark_mean'].values
        
        
        print(model, mcu)
        display(df_filtered_energy[['model_type', 'cmsis-nn','latency_mean']])#, 'tflu_benchmark']])
        
        display(df_filtered_benchmark[['model_type', 'cmsis-nn','tflu_mcu_benchmark_mean','tflu_mcu_benchmark_mean_50']])


01d_ResNet20_CIFAR-10 NUCLEO_F767ZI


,model_type,cmsis-nn,latency_mean
0,W-float32_A-float32,cmsis-nn,1.60724152
0,W-float32_A-float32,none,1.60776026
0,W-int8_A-int8,cmsis-nn,0.44809718
0,W-int8_A-int8,none,1.11399445


,model_type,cmsis-nn,tflu_mcu_benchmark_mean,tflu_mcu_benchmark_mean_50
3,W-float32_A-float32,cmsis-nn,NaN,1613397.56
2,W-float32_A-float32,none,NaN,1613537.54
1,W-int8_A-int8,cmsis-nn,NaN,449302.14
0,W-int8_A-int8,none,NaN,1119745.08


01d_ResNet20_CIFAR-10 DISCO_F469NI


,model_type,cmsis-nn,latency_mean
0,W-float32_A-float32,cmsis-nn,6.13124086
0,W-float32_A-float32,none,6.13123500
0,W-int8_A-int8,cmsis-nn,0.98323892
0,W-int8_A-int8,none,6.48632022


...

,model_type,cmsis-nn,latency_mean
0,W-float32_A-float32,cmsis-nn,0.05935484
0,W-float32_A-float32,none,0.05839063
0,W-int8_A-int8,cmsis-nn,0.01608033
0,W-int8_A-int8,none,0.06546622


,model_type,cmsis-nn,tflu_mcu_benchmark_mean,tflu_mcu_benchmark_mean_50
13,W-float32_A-float32,cmsis-nn,59508.9206,NaN
12,W-float32_A-float32,none,58543.0699,NaN
15,W-int8_A-int8,cmsis-nn,16106.0121,NaN
14,W-int8_A-int8,none,65597.8391,NaN


LeNet-MNIST NUCLEO_L496ZG


,model_type,cmsis-nn,latency_mean
0,W-float32_A-float32,cmsis-nn,0.11301297
0,W-float32_A-float32,none,0.11206141
0,W-int8_A-int8,cmsis-nn,0.03646437
0,W-int8_A-int8,none,0.12052125


,model_type,cmsis-nn,tflu_mcu_benchmark_mean,tflu_mcu_benchmark_mean_50
1,W-float32_A-float32,cmsis-nn,113151.6721,NaN
0,W-float32_A-float32,none,112196.2348,NaN
3,W-int8_A-int8,cmsis-nn,36391.3420,NaN
2,W-int8_A-int8,none,120669.5127,NaN


## Energy for each model

### hue = cmsis-nn

In [9]:
hue = 'cmsis-nn'

mcus = df['MCU'].unique()
models = df['model_name'].unique()

for model in models:
    for mcu in mcus:



        filter = (df['weights'] == df['activations']) & (df['MCU'] == mcu) & (df['model_name'] == model)


        df_filtered = df.loc[filter]
        model_name = get_model_name(df_filtered['model'].unique()[0])

        # let's sort the filtered df so that the order in the plots match
        df_filtered = df_filtered.sort_values(by=['model_type','cmsis-nn'])
        

        df_filtered.loc[:,('energy_mean')] *= 1000


        fig = sns.catplot(y='model_type', x='energy_mean', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9)
        display(df_filtered[['model_type', 'energy_mean']].drop_duplicates())



        plt.title(mcu + ' - ' + model_name)

        plt.xlabel("Inference Energy Consumption [mJ]")
        plt.ylabel("Model Type")

        fig.set_yticklabels(['\\texttt{float32}', '\\texttt{int8}'])

        fig._legend.texts[0].set_text("\\texttt{cmsis-nn}")
        fig._legend.texts[1].set_text("\\textit{none}")
        
        
        filename = f'figures/energy/perMCU/{mcu}_{model_name}_hue-{hue}'

        print(filename)

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,energy_mean
16,W-float32-A-float32,638.23991973
0,W-float32-A-float32,633.66769753
5,W-int8-A-int8,193.27060836
3,W-int8-A-int8,584.32011147


figures/energy/perMCU/F7_ResNet20_CIFAR-10_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,energy_mean
15,W-float32-A-float32,1244.36935550
20,W-float32-A-float32,1283.69909146
10,W-int8-A-int8,223.95316562
21,W-int8-A-int8,1279.32453491


...

,model_type,energy_mean
8,W-float32-A-float32,12.78852949
1,W-float32-A-float32,11.94871866
6,W-int8-A-int8,3.74759469
13,W-int8-A-int8,14.65044108


figures/energy/perMCU/F4_LeNet-MNIST_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,energy_mean
4,W-float32-A-float32,5.10258974
12,W-float32-A-float32,5.09570572
11,W-int8-A-int8,1.58983832
9,W-int8-A-int8,5.43261917


figures/energy/perMCU/L4_LeNet-MNIST_hue-cmsis-nn


basically all other plots are not interesting if i can show that it's the same as the latency

### Latency Energy Linear Regression

In [10]:
model_name = get_model_name(df['model'].unique()[0])


df_filtered = df.copy()

# let's sort the filtered df so that the order in the plots match
df_filtered = df_filtered.sort_values(by=['model_type'])


df_filtered.loc[:,('energy_mean')] *= 1000


fig = sns.lmplot(x="latency_mean", y="energy_mean", hue="MCU", data=df_filtered, height=4, aspect=16/9,  hue_order=["L4", "F4", "F7"])



#plt.title('Regression across all models, all MCUs')

plt.xlabel("Inference Latency [s]")
plt.ylabel("Inference Energy Consumption [mJ]")




filename = f'figures/energy/regression/regression'

print(filename)
plt.legend()

plt.tight_layout()
plt.savefig(filename + '.pdf')
tikzplotlib.clean_figure()
tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/energy/regression/regression


### Linear regression for models individually

In [11]:
models = df['model_name'].unique()

for model in models:


    filter = (df['weights'] == df['activations']) & (df['model_name'] == model)


    df_filtered = df.loc[filter]




    model_name = get_model_name(df_filtered['model'].unique()[0])

    # let's sort the filtered df so that the order in the plots match
    df_filtered = df_filtered.sort_values(by=['model_type'])


    df_filtered.loc[:,('energy_mean')] *= 1000


    fig = sns.lmplot(x="latency_mean", y="energy_mean", hue="MCU", data=df_filtered, height=4, aspect=16/9)
    display(df_filtered[['model_type','latency_mean', 'energy_mean']])



    plt.title(model_name)

    plt.xlabel("Inference Latency [s]")
    plt.ylabel("Inference Energy [mJ]")


    filename = f'figures/energy/regression/regression_{model_name}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
0,W-float32-A-float32,1.60776026,633.66769753
15,W-float32-A-float32,6.13124086,1244.36935550
16,W-float32-A-float32,1.60724152,638.23991973
20,W-float32-A-float32,6.13123500,1283.69909146
3,W-int8-A-int8,1.11399445,584.32011147
5,W-int8-A-int8,0.44809718,193.27060836
10,W-int8-A-int8,0.98323892,223.95316562
14,W-int8-A-int8,2.20918812,98.19830809
17,W-int8-A-int8,12.04171437,550.70729805
21,W-int8-A-int8,6.48632022,1279.32453491


figures/energy/regression/regression_ResNet20_CIFAR-10


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
1,W-float32-A-float32,0.05839063,11.94871866
4,W-float32-A-float32,0.11301297,5.10258974
7,W-float32-A-float32,0.01995101,8.38124693
8,W-float32-A-float32,0.05935484,12.78852949
12,W-float32-A-float32,0.11206141,5.09570572
18,W-float32-A-float32,0.01958125,8.18971318
2,W-int8-A-int8,0.01791730,8.43169922
6,W-int8-A-int8,0.01608033,3.74759469
9,W-int8-A-int8,0.12052125,5.43261917
11,W-int8-A-int8,0.03646437,1.58983832


figures/energy/regression/regression_LeNet-MNIST


#### Dot plot for models individually

In [12]:
models = df['model_name'].unique()

for model in models:


    filter = (df['weights'] == df['activations']) & (df['model_name'] == model)


    df_filtered = df.loc[filter]




    model_name = get_model_name(df_filtered['model'].unique()[0])

    # let's sort the filtered df so that the order in the plots match
    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])


    df_filtered.loc[:,('energy_mean')] *= 1000


    fig = sns.relplot(x="latency_mean", y="energy_mean", hue="MCU", data=df_filtered, height=4, aspect=16/9)
    display(df_filtered[['model_type','latency_mean', 'energy_mean']])



    plt.title(model_name)

    plt.xlabel("Inference Latency [s]")
    plt.ylabel("Inference Energy Consumption [mJ]")


    filename = f'figures/energy/regression/scatter_{model_name}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
#     tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
15,W-float32-A-float32,6.13124086,1244.36935550
20,W-float32-A-float32,6.13123500,1283.69909146
0,W-float32-A-float32,1.60776026,633.66769753
16,W-float32-A-float32,1.60724152,638.23991973
10,W-int8-A-int8,0.98323892,223.95316562
21,W-int8-A-int8,6.48632022,1279.32453491
3,W-int8-A-int8,1.11399445,584.32011147
5,W-int8-A-int8,0.44809718,193.27060836
14,W-int8-A-int8,2.20918812,98.19830809
17,W-int8-A-int8,12.04171437,550.70729805


figures/energy/regression/scatter_ResNet20_CIFAR-10


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
1,W-float32-A-float32,0.05839063,11.94871866
8,W-float32-A-float32,0.05935484,12.78852949
7,W-float32-A-float32,0.01995101,8.38124693
18,W-float32-A-float32,0.01958125,8.18971318
4,W-float32-A-float32,0.11301297,5.10258974
12,W-float32-A-float32,0.11206141,5.09570572
6,W-int8-A-int8,0.01608033,3.74759469
13,W-int8-A-int8,0.06546622,14.65044108
2,W-int8-A-int8,0.01791730,8.43169922
19,W-int8-A-int8,0.00808039,3.21706050


figures/energy/regression/scatter_LeNet-MNIST


## Dot plots including pareto front

In [53]:
models = df['model_name'].unique()

for model in models:


    filter = (df['weights'] == df['activations']) & (df['model_name'] == model)


    df_filtered = df.loc[filter]



    model_name = get_model_name(df_filtered['model'].unique()[0])

    # let's sort the filtered df so that the order in the plots match
    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])


    df_filtered.loc[:,('energy_mean')] *= 1000
    
    
        # determining pareto points
    x_index = df_filtered['latency_mean'].idxmin()
    y_index = df_filtered['energy_mean'].idxmin()
    
    
    x_values = df_filtered.loc[[x_index,y_index,y_index],'latency_mean']
    y_values = df_filtered.loc[[x_index,x_index,y_index],'energy_mean']
    print(x_values[0:2])
    print(y_values)
    

    fig, ax = plt.subplots(figsize=(5,4))

    plt.plot(x_values, y_values, c='r', ls='-', label='Pareto Frontier');

    sns.scatterplot(ax=ax, x="latency_mean", y="energy_mean", hue="MCU", data=df_filtered, hue_order=["L4", "F4", "F7"], legend='brief')
    display(df_filtered[['model_type','latency_mean', 'energy_mean']])



    

    #sns.lineplot(ax=ax, x=x_values[0:2], y=y_values[0:2], c='r', ls='--', label='Pareto Frontier');
    
    #sns.lineplot(ax=ax, x=x_values[1:2], y=y_values[1:2], c='r', ls='--', label='Pareto Frontier');

    #sns.lineplot(ax=ax, x=p1, y=[p2.iloc[0],5], c='g', ls='--', label='Pareto Frontier');

    #plt.legend()
    #plt.legend(['Pareto Frontier', 'L4', 'F4', 'F7'])
    #plt.title(model_name)

    plt.xlabel("Inference Latency [s]")
    plt.ylabel("Inference Energy Consumption [mJ]")



    
    filename = f'figures/energy/regression/scatter_pareto_{model_name}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
#     tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

5     0.44809718
14    2.20918812
Name: latency_mean, dtype: float64
5     193.27060836
5     193.27060836
14     98.19830809
Name: energy_mean, dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
15,W-float32-A-float32,6.13124086,1244.36935550
20,W-float32-A-float32,6.13123500,1283.69909146
0,W-float32-A-float32,1.60776026,633.66769753
16,W-float32-A-float32,1.60724152,638.23991973
10,W-int8-A-int8,0.98323892,223.95316562
21,W-int8-A-int8,6.48632022,1279.32453491
3,W-int8-A-int8,1.11399445,584.32011147
5,W-int8-A-int8,0.44809718,193.27060836
14,W-int8-A-int8,2.20918812,98.19830809
17,W-int8-A-int8,12.04171437,550.70729805


figures/energy/regression/scatter_pareto_ResNet20_CIFAR-10
19    0.00808039
11    0.03646437
Name: latency_mean, dtype: float64
19    3.21706050
19    3.21706050
11    1.58983832
Name: energy_mean, dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
1,W-float32-A-float32,0.05839063,11.94871866
8,W-float32-A-float32,0.05935484,12.78852949
7,W-float32-A-float32,0.01995101,8.38124693
18,W-float32-A-float32,0.01958125,8.18971318
4,W-float32-A-float32,0.11301297,5.10258974
12,W-float32-A-float32,0.11206141,5.09570572
6,W-int8-A-int8,0.01608033,3.74759469
13,W-int8-A-int8,0.06546622,14.65044108
2,W-int8-A-int8,0.01791730,8.43169922
19,W-int8-A-int8,0.00808039,3.21706050


figures/energy/regression/scatter_pareto_LeNet-MNIST


In [14]:

filter = (df['weights'] == df['activations'])

df_filtered = df.loc[filter]




model_name = get_model_name(df_filtered['model'].unique()[0])

# let's sort the filtered df so that the order in the plots match
df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])


df_filtered.loc[:,('energy_mean')] *= 1000


fig = sns.relplot(x="latency_mean", y="energy_mean", hue="MCU", col="model_name", data=df_filtered, height=4, aspect=16/9)
display(df_filtered[['model_type','latency_mean', 'energy_mean']])



plt.title(model_name)

plt.xlabel("Inference Latency [s]")
plt.ylabel("Inference Energy [mJ]")


filename = f'figures/energy/regression/scatter_column'

print(filename)

plt.tight_layout()
plt.show()
plt.savefig(filename + '.pdf')
# tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,latency_mean,energy_mean
1,W-float32-A-float32,0.05839063,11.94871866
8,W-float32-A-float32,0.05935484,12.78852949
15,W-float32-A-float32,6.13124086,1244.36935550
20,W-float32-A-float32,6.13123500,1283.69909146
0,W-float32-A-float32,1.60776026,633.66769753
7,W-float32-A-float32,0.01995101,8.38124693
16,W-float32-A-float32,1.60724152,638.23991973
18,W-float32-A-float32,0.01958125,8.18971318
4,W-float32-A-float32,0.11301297,5.10258974
12,W-float32-A-float32,0.11206141,5.09570572


figures/energy/regression/scatter_column
